###**DDoS Attack Prediection by Random Forest Decision Tree**


In [1]:

%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit

from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, GBTClassificationModel 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [2]:
%pyspark
PYSPARK_CLI = True
if PYSPARK_CLI:
	sc = SparkContext.getOrCreate()
	spark = SparkSession(sc)

####**To Read the dataset in csv format from HDFS and To Infer the Schema**


In [4]:
%pyspark
# File location and type
file_location = "/user/fmamagh/project/final_dataset.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

####**To get information from the dataframe (df1)**


In [6]:
%pyspark
df1.printSchema()

In [7]:
%pyspark
print(len(df1.columns))
print(df1.columns)


In [8]:
%pyspark

df1.groupBy("Label").count().show()

####**To clean the dataframe**

The columns of the Source IPs , the Destination IPs and the Flow IDs have around 4500 distinct values and do not yeild any specific pattern for prediction.
The columns that are not used as features are dropped.


In [10]:
%pyspark
print("type of the column:", df1.schema["Flow ID"].dataType)
print("Number of distinct values in a column:", df1.select("Dst IP").distinct().count())

In [11]:
%pyspark
df_clean1 = df1.drop("_c0", "Dst IP", "Flow Byts/s", "Flow ID", "Flow Pkts/s", "Src IP", "Timestamp")

####**To sample the dataframe based on equal fraction of values in the Label column**


In [13]:
%pyspark
v1_seed=5561
v2_seed=5562
sample_dateset_1 = df1.sampleBy(col("Label"), fractions={"Benign": 0.007,"ddos": 0.007}, seed=v1_seed)

In [14]:
%pyspark
sample_dateset_1.groupBy("Label").count().show()

The datatype of the Label column is changed from string to double and the value of "ddos" is changed to "1.0" and the value of "Benign" is changed to "0.0"


In [16]:
%pyspark
#sampled = sample_dateset_1.select("Src Port","Dst Port","Protocol","Flow Duration","Tot Fwd Pkts","Tot Bwd Pkts","TotLen Fwd Pkts","TotLen Bwd Pkts","Fwd Pkt Len Max","Fwd Pkt Len Min","Fwd Pkt Len Mean","Fwd Pkt Len Std","Bwd Pkt Len Max","Bwd Pkt Len Min","Bwd Pkt Len Mean","Bwd Pkt Len Std","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Tot","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Tot","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Len","Bwd Header Len","Fwd Pkts/s","Bwd Pkts/s","Pkt Len Min","Pkt Len Max","Pkt Len Mean","Pkt Len Std","Pkt Len Var","FIN Flag Cnt","SYN Flag Cnt","RST Flag Cnt","PSH Flag Cnt","ACK Flag Cnt","URG Flag Cnt","CWE Flag Count","ECE Flag Cnt","Down/Up Ratio","Pkt Size Avg","Fwd Seg Size Avg","Bwd Seg Size Avg","Fwd Byts/b Avg","Fwd Pkts/b Avg","Fwd Blk Rate Avg","Bwd Byts/b Avg","Bwd Pkts/b Avg","Bwd Blk Rate Avg","Subflow Fwd Pkts","Subflow Fwd Byts","Subflow Bwd Pkts","Subflow Bwd Byts","Init Fwd Win Byts","Init Bwd Win Byts","Fwd Act Data Pkts","Fwd Seg Size Min","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min",(col("Label") == "ddos").cast("Double").alias("label"))

df_clean = df_clean1.select("Src Port","Dst Port","Protocol","Flow Duration","Tot Fwd Pkts","Tot Bwd Pkts","TotLen Fwd Pkts","TotLen Bwd Pkts","Fwd Pkt Len Max","Fwd Pkt Len Min","Fwd Pkt Len Mean","Fwd Pkt Len Std","Bwd Pkt Len Max","Bwd Pkt Len Min","Bwd Pkt Len Mean","Bwd Pkt Len Std","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Tot","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Tot","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Len","Bwd Header Len","Fwd Pkts/s","Bwd Pkts/s","Pkt Len Min","Pkt Len Max","Pkt Len Mean","Pkt Len Std","Pkt Len Var","FIN Flag Cnt","SYN Flag Cnt","RST Flag Cnt","PSH Flag Cnt","ACK Flag Cnt","URG Flag Cnt","CWE Flag Count","ECE Flag Cnt","Down/Up Ratio","Pkt Size Avg","Fwd Seg Size Avg","Bwd Seg Size Avg","Fwd Byts/b Avg","Fwd Pkts/b Avg","Fwd Blk Rate Avg","Bwd Byts/b Avg","Bwd Pkts/b Avg","Bwd Blk Rate Avg","Subflow Fwd Pkts","Subflow Fwd Byts","Subflow Bwd Pkts","Subflow Bwd Byts","Init Fwd Win Byts","Init Bwd Win Byts","Fwd Act Data Pkts","Fwd Seg Size Min","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min",(col("Label") == "ddos").cast("Double").alias("label"))

The list of columns in the new dataframe is double checked.


In [18]:
%pyspark
#sampled.columns
df_cleandf_clean.columns

####**To split the dataframe between train and test dataframes with the ratio of 70% and 30%, repectively.**



In [20]:
%pyspark
splits = df_clean.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

The contents of the new train dataframe is double checked.


In [22]:

%pyspark
train.schema["Src Port"].dataType

In [23]:
%pyspark
featuresList= train.columns
print(featuresList)
print(len(featuresList))
print(train.schema[featuresList[0]].dataType)


A VectorAssembler that combines categorical features into a single vector
A VectorIndexer that creates indexes for a vector of categorical features


In [25]:
%pyspark

#strIdx1 = StringIndexer(inputCol = "Src IP", outputCol = "SrcIPIdx")
#strIdx2 = StringIndexer(inputCol = "Dst IP", outputCol = "DstIPIdx")

catVect = VectorAssembler(inputCols = ["Src Port","Dst Port","Protocol","Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Len", "Bwd Header Len"],handleInvalid="skip", outputCol="catFeatures")

catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")


In [26]:

%pyspark
sample_dateset_1.select("Bwd Header Len").distinct().count()

A VectorAssembler that creates a vector of continuous numeric features
A MinMaxScaler that normalizes continuous numeric features


In [28]:

%pyspark
numVect = VectorAssembler(inputCols = ["Flow Duration", "Tot Fwd Pkts", "Tot Bwd Pkts", "TotLen Fwd Pkts", "TotLen Bwd Pkts", "Fwd Pkt Len Max", "Fwd Pkt Len Min", "Fwd Pkt Len Mean", "Fwd Pkt Len Std", "Bwd Pkt Len Max", "Bwd Pkt Len Min", "Bwd Pkt Len Mean", "Bwd Pkt Len Std", "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Tot", "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Tot", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd Pkts/s", "Bwd Pkts/s", "Pkt Len Min", "Pkt Len Max", "Pkt Len Mean", "Pkt Len Std", "Pkt Len Var", "FIN Flag Cnt", "SYN Flag Cnt", "RST Flag Cnt", "PSH Flag Cnt", "ACK Flag Cnt", "URG Flag Cnt", "CWE Flag Count", "ECE Flag Cnt", "Down/Up Ratio", "Pkt Size Avg", "Fwd Seg Size Avg", "Bwd Seg Size Avg", "Fwd Byts/b Avg", "Fwd Pkts/b Avg", "Fwd Blk Rate Avg", "Bwd Byts/b Avg", "Bwd Pkts/b Avg", "Bwd Blk Rate Avg", "Subflow Fwd Pkts", "Subflow Fwd Byts", "Subflow Bwd Pkts", "Subflow Bwd Byts", "Init Fwd Win Byts", "Init Bwd Win Byts", "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Mean", "Active Std", "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min"], handleInvalid="skip",outputCol="numFeatures")

minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

####**Some of the parameters of Random Forest Decision Tree**

*featureSubsetStrategy* : The number of features to consider for splits at each tree node. Possible values are : 'all' : If numTrees > 1 (forest) : sqrt, all, 'onethird',
'log2' , 'n' (0, 1.0], default = 'auto'

*impurity* = Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'. default: gini

*maxBins* = Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. default:32

*maxDepth* = Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30]. default: 5

*subsamplingRate* = Fraction of the training data used for learning each decision tree, in range (0, 1].

*thresholds* = Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, 
excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold.


#####**class pyspark.ml.classification.RandomForestClassifier**
(*, featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', 
rawPredictionCol='rawPrediction', maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0, maxMemoryInMB=256, 
cacheNodeIds=False, checkpointInterval=10, impurity='gini', numTrees=20, featureSubsetStrategy='auto', seed=None, 
subsamplingRate=1.0, leafCol='', minWeightFractionPerNode=0.0, weightCol=None, bootstrap=True)

[Apache Spark RandomForestClassifier API Reference](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html)



In [30]:

%pyspark
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
rfc = RandomForestClassifier(labelCol="label", featuresCol="features",maxBins=520)

pipeline = Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, rfc])


####**To build the model by TrainValidationSplit and Train Ratio split of 80%**


In [32]:
%pyspark
paramGridCV = ParamGridBuilder() \
  .addGrid(rfc.maxDepth, [10, 15]) \
  .addGrid(rfc.numTrees, [20]) \
  .build()


In [33]:

%pyspark
tvs = TrainValidationSplit(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGridCV, trainRatio=0.8)
#estimatorParamMaps=paramGrid,

In [34]:

%pyspark
# pipeline we implement can train a model
rfcModel = tvs.fit(train)
print ("Model complete!")

####**To Transform the Test dataframe**


In [36]:
%pyspark
prediction = rfcModel.transform(test)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")

predicted.show(20, truncate=True)


####**To calculate the evaluation metrics: Precision, Recall and AUC**


In [38]:

%pyspark
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])

metrics.show()

In [39]:
%pyspark
prediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(20, truncate=True)

In [40]:
%pyspark
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)
print("AUC = ", auc)

In [41]:

%md
####**Feature Importance**

In [42]:
%pyspark
rfcModel = pipeline.fit(train)
print ("Model complete!")

In [43]:
%pyspark
rfcModel.stages[-1].featureImportances

In [44]:
%pyspark
def get_feature_imp(col,imp):
    ft_data_v1=[]
    tmp_c=0
    ft_data = list(zip(col,imp))
    for c1,c2 in ft_date:
        ft_data_v1.insert(tmp_c,[c1,float(c2)])
        tmp_c=tmp_c+1
    ft_df = spark.createDataFrame(data=ft_data_v1,schema=['Column Name','Features'])
    return ft_df 


In [45]:
%pyspark
ft_df_model3=get_feature_imp(featuresList,rfcModel.stages[-1].featureImportances)
ft_df_model4 = ft_df_model.select("Column Name",round(col("Features"),4)).withColumnRenamed("round(Features, 4)", "Features Importance").sort(desc("Features Importance"))
ft_df_model4.show(80)
